In [1]:
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint


from keys import Trip_advisor_key
from keys import geoapify_key

In [4]:

# Definizione dei distretti e delle coordinate
Districts_df = pd.DataFrame({
    'Location': ['Toronto Distillery District', 'Toronto GTA Area', 'Toronto South West', 'Toronto North West', 'Toronto Yorkville', 'Ottawa City Centre' , 'Niagara Falls'],
    'Lat': [43.6503, 43.6426, 43.642567, 43.6552, 43.6707 , 45.4236, 43.087586], 
    'Lng': [-79.3596, 79.3871, -79.387054, -79.4139,  -79.3930 , -75.7009, -79.081206]  
})


# Definizione del dataframe dei ristoranti
Restaurants_df = pd.DataFrame(columns=['Location', 'Restaurant Name', 'Postal Code', 'Restaurant Address', 'Lat', 'Lng'])

# Preparazione dello script per il primo distretto scelto
radius = 1000

params = {
    'categories': 'catering.restaurant',
    'apiKey': geoapify_key,
}

base_url = 'https://api.geoapify.com/v2/places'

for index, row in Districts_df.iterrows():
    latitude = row['Lat']
    longitude = row['Lng']

    params['filter'] = f'circle:{longitude},{latitude},{radius}'
    params['bias'] = f'proximity:{longitude},{latitude}'

    response = requests.get(base_url, params=params).json()
    for restaurant in response.get('features', []):
        try:
            
            new_row = {
                            'Location': row['Location'],
                            'Restaurant Name': restaurant["properties"]["address_line1"],
                            'Postal Code' : restaurant["properties"]["postcode"],
                            'Restaurant Address': restaurant["properties"]["address_line2"],
                            'Lat': restaurant["geometry"]["coordinates"][1],
                            'Lng': restaurant["geometry"]["coordinates"][0]
                        }
            
            Restaurants_df.loc[len(Restaurants_df)] = new_row
        except(KeyError, IndexError):
            print('Errore nella gestione dei dati del ristorante.')


In [5]:
Restaurants_df.head()

,Location,Restaurant Name,Postal Code,Restaurant Address,Lat,Lng
0,Toronto Distillery District,Archeo Trattoria,M5A 3C4,"31 Trinity Street, Old Toronto, ON M5A 3C4, Ca...",43.650682,-79.359630
1,Toronto Distillery District,El Catrin,M5A 3C4,"Mill Street, Old Toronto, ON M5A 3C4, Canada",43.650693,-79.359245
2,Toronto Distillery District,Oyster House & grill,M5A 3C4,"Tank House Lane, Old Toronto, ON M5A 3C4, Canada",43.650350,-79.358811
3,Toronto Distillery District,Pure Spirits,M5A 3C4,"17 Tank House Lane, Old Toronto, ON M5A 3C4, C...",43.650395,-79.358754
4,Toronto Distillery District,The Fermenting Cellar,M5A 3C4,"28 Distillery Lane, Old Toronto, ON M5A 3C4, C...",43.649806,-79.360538


In [6]:
# Export the City_Data into a csv
Restaurants_df.to_csv("output_data/restaurants.csv", index_label="Restaurant ID")

In [7]:
Restaurants_df['Location ID'] = ''


for index, restaurant in Restaurants_df.iterrows():
    
    url = f"https://api.content.tripadvisor.com/api/v1/location/search?key={Trip_advisor_key}&searchQuery={restaurant['Restaurant Name']}&language=en"

    headers = {"accept": "application/json"}

    response = requests.get(url, headers=headers)
    
    for res in response['data']:
        
        if 
        #response['data'][res]['location_id']

#print(response.text)
Restaurants_df

SyntaxError: invalid syntax (3285224266.py, line 14)

In [14]:
url = f"https://api.content.tripadvisor.com/api/v1/location/search?key={Trip_advisor_key}&searchQuery=Archeo Trattoria&language=en"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)
    
print(response.text)

{
  "data": [
    {
      "location_id": "5078107",
      "name": "Archeo",
      "address_obj": {
        "street1": "31 Trinity St",
        "city": "Toronto",
        "state": "Ontario",
        "country": "Canada",
        "postalcode": "M5A 3C4",
        "address_string": "31 Trinity St, Toronto, Ontario M5A 3C4 Canada"
      }
    },
    {
      "location_id": "8075561",
      "name": "Archeo Diving",
      "address_obj": {
        "street1": "Via Degli Oleandri 10",
        "street2": "Tourist Port of Villasimius",
        "city": "Villasimius",
        "state": "Province of Cagliari",
        "country": "Italy",
        "postalcode": "09049",
        "address_string": "Via Degli Oleandri 10 Tourist Port of Villasimius, 09049, Villasimius, Sardinia Italy"
      }
    },
    {
      "location_id": "1102359",
      "name": "Trattoria Za Za",
      "address_obj": {
        "street1": "Piazza del Mercato Centrale 26R",
        "street2": "",
        "city": "Florence",
        "stat

In [2]:
radius = 1000

params = {
    'categories': 'catering.restaurant',
    'apiKey': geoapify_key,
}

base_url = 'https://api.geoapify.com/v2/places'

latitude = 43.6503
longitude = -79.3596

params['filter'] = f'circle:{longitude},{latitude},{radius}'
params['bias'] = f'proximity:{longitude},{latitude}'

response = requests.get(base_url, params=params).json()

pprint(response)

{'features': [{'geometry': {'coordinates': [-79.3596299, 43.65068200054068],
                            'type': 'Point'},
               'properties': {'address_line1': 'Archeo Trattoria',
                              'address_line2': '31 Trinity Street, Old '
                                               'Toronto, ON M5A 3C4, Canada',
                              'categories': ['catering',
                                             'catering.restaurant',
                                             'wheelchair',
                                             'wheelchair.limited'],
                              'city': 'Old Toronto',
                              'country': 'Canada',
                              'country_code': 'ca',
                              'county': 'Golden Horseshoe',
                              'datasource': {'attribution': '© OpenStreetMap '
                                                            'contributors',
                                    